<a href="https://colab.research.google.com/github/psaw/hse-ai24-ml/blob/main/KNN_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Метрические алгоритмы. Практика

В этом домашнем задании вы будете решать задачу классификации бутылок вина по различным характеристикам.

## Импорт библиотек, установка константных значений

In [14]:
import pandas as pd
import numpy as np

In [15]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [16]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [17]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

X.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


## Задание 1

Посмотрите на количество классов и количество объектов каждого класса в датасете.

**Вопрос**:  
Сколько классов в задаче?  
**Ответ**:
0 - 59, 1 - 71, 2 - 48

In [18]:
# ваш код здесь
print(y.value_counts())

target
1    71
0    59
2    48
Name: count, dtype: int64


## Задание 2

Мы имеем дело с многоклассовой классификацией. Кроме того, классы не очень хорошо сбалансированы, поэтому для оценки качества модели метрика *accuracy* не подойдет.

Разбейте данные на тренировочную и тестовую части:  
тестовая часть - 25% от всех данных, зафиксируйте `random_state = RANDOM_STATE`.

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_SIZE, random_state=RANDOM_STATE)

**Вопрос:**

Все ли признаки в данных одного масштаба?  
Проверьте это, выведя основные числовые характеристики матрицы `X_train` методом `describe` из библиотеки `pandas`.

По полученной таблице числовых характеристик определите, какой признак измеряется в сотнях?  
(если вариантов несколько, выберите признак с наибольшим средним значением).

**Ответ:**  `proline`

In [20]:
# ваш код здесь
X_train.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,12.972857,2.386842,2.362180,19.433835,100.759398,2.277068,2.021203,0.363534,1.608647,5.017594,0.959444,2.610000,742.992481
std,0.829993,1.098905,0.280606,3.467312,14.999571,0.645696,1.005537,0.126923,0.576964,2.202516,0.234545,0.729961,306.867593
min,11.030000,0.890000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.420000,1.740000,0.480000,1.270000,278.000000
25%,12.290000,1.640000,2.210000,17.200000,88.000000,1.700000,1.200000,0.260000,1.250000,3.250000,0.780000,1.830000,500.000000
50%,12.990000,1.900000,2.360000,19.400000,98.000000,2.230000,2.140000,0.340000,1.560000,4.800000,0.970000,2.810000,675.000000
75%,13.690000,3.170000,2.540000,21.500000,108.000000,2.800000,2.880000,0.450000,1.960000,6.130000,1.120000,3.200000,970.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,10.800000,1.710000,4.000000,1547.000000


## Задание 3

KNN требует того, чтобы все признаки были одного масштаба, поэтому масштабируйте данные при помощи `StandardScaler`.

Напоминаем, что обучать метод нужно только по тренировочным данным, а применять и к трейну, и к тесту.

После применения `StandardScaler` преобразуйте `X_train` и `X_test` к типу `pd.DataFrame`, названия новых объектов оставьте `X_train` и `X_test`.

In [21]:
from sklearn.preprocessing import StandardScaler

# ваш код здесь
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
X_train.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0.226328,-0.024519,1.101127,-0.241394,0.016101,0.812935,1.216668,-0.502463,2.090070,0.301887,0.301960,0.770065,1.445831
1,-1.079807,-0.280281,-2.368742,-0.559842,-0.251581,-0.073170,0.148537,-0.818807,-0.362996,-0.782781,1.329088,0.481291,-0.107920
2,-0.668617,1.866297,1.315759,2.045642,0.083022,-0.119807,0.108607,0.525654,0.176331,-1.338787,-0.168807,0.701309,-1.236434
3,-0.475115,-1.001897,-0.937868,0.163904,0.149942,-1.239098,-1.438686,1.316514,-0.362996,1.176913,-1.623905,-1.457623,-0.336894
4,1.036615,-0.673059,0.850725,-0.646691,-0.452343,0.268836,0.967105,-1.135151,1.185393,0.265427,1.200697,1.045088,1.707515


Обучите KNN с параметрами по умолчанию на тренировочных данных и сделайте предсказание на тесте.

In [22]:
from sklearn.neighbors import KNeighborsClassifier

# ваш код здесь
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)


Будем измерять качество модели по метрике **weighted** $f1$-score.

Чтобы выбрать тип усреднения (micro, macro, weighted) в функции `f1_score` необходимо задать этот тип в гиперпараметре `average`.

Вычислите $f1$-score на тестовых данных.

**Вопрос:**

Чему равен $f1$-score на тестовых данных?

**Ответ:**

f1 (weighted) = 0.9550512333965844

In [32]:
from sklearn.metrics import f1_score

# ваш код здесь
print(f"f1 (weighted) = {f1_score(y_test, y_pred, average='weighted')}")

f1 (weighted) = 0.9550512333965844


## Задание 4

Попробуем улучшить модель.

Подберите оптимальное количество соседей (`n_neighbors`) из диапазона *от 3 до 30 с шагом 2* и веса соседей (`weights`):  
`uniform`, `distance` по кросс-валидации с тремя фолдами на тренировочных данных.

Используйте `GridSearchCV` и метрику `f1_weighted`.

In [37]:
from sklearn.model_selection import GridSearchCV

# ваш код здесь
grid = {
    'n_neighbors': np.arange(3, 20, 2),
    'weights': ['uniform', 'distance'],
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, grid, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_score_)
print(grid_search.best_params_)

# 0.9624902376605162
# {'n_neighbors': np.int64(19), 'weights': 'uniform'}

0.9624902376605162
{'n_neighbors': np.int64(19), 'weights': 'uniform'}


Возьмите best_estimator_, полученный при обучении GridSearchCV и с помощью него  
сделайте предсказание на тесте и вычислите метрику `f1_weighted`.

In [38]:
# ваш код здесь
best_knn = grid_search.best_estimator_
y_pred_best = best_knn.predict(X_test)
f1_best = f1_score(y_test, y_pred_best, average='weighted')
print(f"f1 (weighted) with best estimator = {f1_best}")

f1 (weighted) with best estimator = 0.9550512333965844


**Вопрос:**

Удалось ли при помощи подбора гиперпараметров улучшить качество модели на тестовых данных?

**Ответ:**  НЕТ

## Задание 5

Выведите на экран матрицу ошибок.

Используйте модель с подобранными при помощи `GridSearch` гиперпараметрами.


**Вопрос:**  
По этой матрице определите, какие классы между собой путает модель?

**Ответ:**  
"0 и 1" и "1 и 2"  (по второй строке таблицы - это про реальный 1 класс, 1 раз его определили как 0, и 1 раз - как 2)

In [39]:
from sklearn.metrics import confusion_matrix

# ваш код здесь
confusion_matrix(y_test, y_pred_best)

array([[15,  0,  0],
       [ 1, 16,  1],
       [ 0,  0, 12]])

## Бонус (эксперименты с LSH)

Скопируйте все функции из [ноутбука в уроке "Быстрый поиск соседей"](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH).

In [40]:
# ваш код здесь
def knn_search(query, data, k=5):

    dists = np.sqrt(np.sum((data - query) ** 2, axis=1))  # вычисляем расстояния от объекта query до всех точек датасета
    inds = np.argsort(dists)  # сортируем по возрастанию расстояний
    inds_k = inds[:k]         # берем top-k точек с наименьшими расстояниями

    return data[inds_k], dists[inds_k]


def generate_hyperplanes(data, bucket_size=16):
    m = data.shape[0]            # число объектов
    n = data.shape[1]            # число признаков
    b = m // bucket_size         # количество корзин
    h = int(np.log2(b))          # количество гиперплоскостей
    H = rng.normal(size=(h, n))  # гиперплоскости, заданные своими нормалями
    return H


def hamming_hash(data, hyperplanes):
    b = len(hyperplanes)
    hash_key = (data @ hyperplanes.T) >= 0

    dec_vals = np.array([2 ** i for i in range(b)], dtype=int)
    hash_key = hash_key @ dec_vals

    return hash_key


def locality_sensitive_hash(data, hyperplanes):
    hash_vals = hamming_hash(data, hyperplanes)
    hash_table = {}
    for i, v in enumerate(hash_vals):
        if v not in hash_table:
            hash_table[v] = set()
        hash_table[v].add(i)

    return hash_table


def approx_knn_search(query, data, k=5, bucket_size=16):
    candidates = set()

    hyperplanes = generate_hyperplanes(data)
    hash_table = locality_sensitive_hash(data, hyperplanes) # формируем хеш-таблицу по датасету

    query_hash = hamming_hash(query, hyperplanes)
    if query_hash in hash_table:
        candidates = candidates.union(hash_table[query_hash])
    candidates = np.stack([data[i] for i in candidates], axis=0) # находим кандидатов = объекты, попадающие с query в одну корзину

    return knn_search(query, candidates, k=k) # с помощью KNN ищем ближайших соседей только из найденных кандидатов

При помощи `knn_search` найдите ближайших соседей к вину `X_test.iloc[0]` в **тренировочных** данных.

Обратите внимание, что функция `knn_search` принимает на вход `np.array`, а не `pd.DataFrame`. Поэтому переведите аргументы в `np.array`, приписав к необходимому объекту $X$: `X.values`.

In [46]:
type(X_test.to_numpy())

numpy.ndarray

In [99]:
%%time

# ваш код здесь
top5, distanses = knn_search(X_test.to_numpy()[0], X_train.to_numpy(), k=5)
print(top5)
print(distanses)

[[ 1.50827494  1.46438454  0.52877788 -1.80468392  1.82295821  1.12384935
   1.01701759 -1.2933229   0.82004316  0.03755595 -0.29719787  1.26510707
   0.05563253]
 [ 0.92777049 -0.65479046 -0.40129023 -0.87828967  1.15375191  0.50202111
   0.87726211 -1.21423695  0.0197522   0.37936305 -0.16880684  0.81131867
   1.03694899]
 [-0.05183075  1.29083141  1.02958379 -0.24139362  0.08302183  0.20665269
   0.38811793 -0.89789314  0.64606686 -0.23588973  0.30196024  1.25135591
   0.08834308]
 [ 1.50827494 -0.62738734 -0.22243098 -0.90723949  1.15375191  1.43476348
   0.97708746 -0.81880719  0.73305501  0.62090673 -0.08321283  0.96258147
   0.74255405]
 [ 1.31477345 -0.69132796 -0.29397468 -0.99408895  1.68911695  1.12384935
   1.14679054 -0.97697909  0.85483842  0.28821449  0.55874228  1.51262802
   0.12105362]]
[1.7873321  1.98096241 2.16010164 2.27097456 2.31179552]
CPU times: user 445 μs, sys: 31 μs, total: 476 μs
Wall time: 459 μs


Выведите на экран признаки объекта `X_test.iloc[0]` и признаки ближайшего найденного соседа.

In [100]:
# ваш код здесь
print(X_test.to_numpy()[0])
print(top5[0])
print(distanses[0])

[ 0.80683207  0.65142516  0.70763713 -1.22568751  1.01991065  0.65747817
  1.00703506 -1.53058075  0.08934271  0.03755595  0.00238119  1.03133729
  0.33367219]
[ 1.50827494  1.46438454  0.52877788 -1.80468392  1.82295821  1.12384935
  1.01701759 -1.2933229   0.82004316  0.03755595 -0.29719787  1.26510707
  0.05563253]
1.7873320988662762


**Вопрос:**

Можно ли сказать, что в тренировочных данных есть вино, почти такое же как `X_test.iloc[0]`? (все признаки почти одинаковые)

Какое расстояние между объектом запроса и первым ближайшим соседом?

**Ответ:**
- нет похожего вина, т.к. расстояние 1.79, а это очень много, т.к. признаки все были нормализованы в [-1, 1]
- расстояние между объектом запроса и первым ближайшим соседом равно 1.7873320988662762.

Теперь найдите ближайшего соседа при помощи `approx_knn_search`.

In [121]:
%%time

# ваш код здесь
approx_knn_search(X_test.to_numpy()[0], X_train.to_numpy(), k=5, bucket_size=16)

CPU times: user 415 μs, sys: 25 μs, total: 440 μs
Wall time: 426 μs


(array([[ 1.50827494,  1.46438454,  0.52877788, -1.80468392,  1.82295821,
          1.12384935,  1.01701759, -1.2933229 ,  0.82004316,  0.03755595,
         -0.29719787,  1.26510707,  0.05563253],
        [ 0.32307837,  1.45525016, -0.25820283, -0.55984164,  0.14994246,
          0.56420393,  0.60773369, -0.34429147,  0.08934271, -0.29969372,
         -0.59677692,  0.53629539, -0.20605186],
        [ 0.86730128,  0.79757516,  0.63609343,  0.16390387,  0.41762498,
         -0.69499826, -1.45865091,  1.07925566, -1.40685343,  0.39303533,
          0.00238119, -1.08634194, -0.20605186],
        [ 0.73426901,  0.17643766,  1.17267119,  1.46664579,  0.28378372,
         -1.13027803, -1.17913995,  0.20931019, -0.11942884,  1.66000032,
         -0.93915298, -1.11384427,  0.02292198],
        [ 0.12957688,  1.38217516, -0.00779988,  0.59815118,  0.81914876,
         -1.34791792, -0.63010057, -0.18611957, -0.81533402,  1.99724999,
         -1.6667021 , -1.76014897, -0.63128899]]),
 array([1.787

Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.  
Запустите последнюю ячейку несколько раз и убедитесь, что ближайший сосед находится верно за несколько запусков.

**Вопрос:**

Запустите `knn_search` и `appox_knn_search` несколько раз и сравните время запусков. Какой из подходов в этой задаче работает быстрее?

In [157]:
RUNS = 400

X_0 = X_test.to_numpy()[0]
X_t = X_train.to_numpy()

In [170]:
%%time
# Measure time for knn_search
for _ in range(RUNS):
    knn_search(X_0, X_t, k=5)

# CPU times: user 6.37 ms, sys: 3.38 ms, total: 9.74 ms
# Wall time: 8.55 ms

CPU times: user 6.37 ms, sys: 3.38 ms, total: 9.74 ms
Wall time: 8.55 ms


In [168]:
%%time
# Measure time for approx_knn_search
for _ in range(RUNS):
    approx_knn_search(X_0, X_t, k=5, bucket_size=16)

# CPU times: user 21.9 ms, sys: 1.97 ms, total: 23.9 ms
# Wall time: 22.6 ms

CPU times: user 21.9 ms, sys: 1.97 ms, total: 23.9 ms
Wall time: 22.6 ms
